In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from collections import deque
import sys
import cv2
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential, optimizers, losses, datasets, metrics, callbacks, models

plt.rcParams['font.family'] = ['SimSun']
plt.rcParams['axes.unicode_minus'] = False

sys.path.append(os.path.join('game'))

# print('Headless Mode')
# os.environ["SDL_VIDEODRIVER"] = "dummy"

from IPython.display import clear_output

print(tf.__version__)

2.4.1


# 定义参数

In [2]:
ACTIONS = 2 # number of valid actions

# 模型存放文件夹路径
MODEL_FOLDER = os.path.join("FlappyBird_Model")
MODEL_DIR = os.path.join(MODEL_FOLDER, "FlappyBird_Model.h5")
MODEL_DIR

'FlappyBird_Model\\FlappyBird_Model.h5'

# 将游戏返回的图片转为80*80的灰阶图片
调整大小 => 灰度化 => 二值化 => 4帧叠加

In [3]:
# preprocess raw image to 80*80 gray image
def preprocess(observation):
    observation = cv2.cvtColor(cv2.resize(observation, (80, 80)), cv2.COLOR_BGR2GRAY)
    ret, observation = cv2.threshold(observation, 1, 255, cv2.THRESH_BINARY)
    return np.reshape(observation,(80,80,1))

# 加载模型

In [4]:
def load_model():
    print('加载模型')
    return models.load_model(MODEL_DIR)

# 获得下一个Action

In [5]:
def getAction(model, state, timeStep):
    state_ = tf.expand_dims(state, axis=0)  # 扩展维度 (80,80,4) => (1,80,80,4)
    QValue = model(state_)[0]
    print("步数", timeStep, 'Q值', QValue.numpy(), end='')

    action = np.zeros(ACTIONS)  # 生成[0, 0]初始action
    action_index = 0

    # 按照Q表采取Action
    action_index = np.argmax(QValue)
    action[action_index] = 1

    return action, action_index

# 打印日志

In [6]:
def print_info(timeStep, action_index, reward, terminal, score, highest_record):
    action_text = ''
    if action_index == 0:
        action_text = '不跳'
    else:
        action_text = '跳'
    if terminal:
        print("===> 步数", timeStep, "/ 本次行为", action_index, action_text, "/ 本次奖励", reward, "/ Game Over", "/ 累计分数", score, '/ 最高得分', highest_record)
#         sys.__stdout__.write("===> 步数"+str(timeStep)+"/ 本次行为"+str(action_index)+"/ 本次奖励"+str(reward)+"/ Game Over"+"/ 累计分数"+str(score)+ '/ 最高得分'+str(highest_record))
    else:
        print("===> 步数", timeStep, "/ 本次行为", action_index, action_text, "/ 本次奖励", reward, "/ Running  ", "/ 当前分数", score, '/ 最高得分', highest_record)
#         sys.__stdout__.write("===> 步数"+str(timeStep)+"/ 本次行为"+str(action_index)+"/ 本次奖励"+str(reward)+"/ Running  "+"/ 当前分数"+str(score)+ '/ 最高得分'+str(highest_record))

# 主流程

In [ ]:
import wrapped_flappy_bird as game

flappyBird = game.GameState()

timeStep = 0 
total_reward = 0
highest_record = 0

# 载入模型
model = load_model()

# 第一次与智能体交互
action = np.array([1, 0])# do nothing
observation0, reward, terminal = flappyBird.frame_step(action)

# 展示第一张图片
plt.imshow(np.rot90(np.rot90(np.rot90(observation0))))
plt.gca().invert_xaxis()
plt.title('The First Picture')
plt.show()

# 处理第一张图片
observation0 = cv2.cvtColor(cv2.resize(observation0, (80, 80)), cv2.COLOR_BGR2GRAY)
ret, observation0 = cv2.threshold(observation0, 1, 255, cv2.THRESH_BINARY)

# 展示第一张图片
plt.imshow(np.rot90(np.rot90(np.rot90(observation0))))
plt.gca().invert_xaxis()
plt.title('The First Picture')
plt.show()

# 第一次的4张图片都是一样的
currentObservation = np.stack((observation0, observation0, observation0, observation0), axis = 2)

while True:
    # 根据上一次的Observation，获得下一次的Action
    action, action_index = getAction(model, currentObservation, timeStep)
    
    # 将Action传入游戏，获得游戏的反馈(状态，奖励，是否GameOver，生效的行为)
    nextObservation, reward, terminal = flappyBird.frame_step(action)

    # 处理图片
    nextObservation = preprocess(nextObservation)
    
    # 将最近的图片后面追加老的前3张图片，从而得到一个新状态
    newObservation = np.append(nextObservation, currentObservation[:,:,:3], axis = 2)
        
    # 计算得分
    if terminal:
        total_reward = 0# Game Over 得分清零
    elif reward == 1:
        total_reward += reward
        
    if highest_record < total_reward:
        highest_record = total_reward

    # 更替状态
    currentObservation = newObservation
    
    # 打印日志
    if timeStep % 1 == 0:
        print_info(timeStep, action_index, reward, terminal, total_reward, highest_record)
        
    # 每第1000次，打印一次图片
    if timeStep % 1000 == 0:
        plt.imshow(np.rot90(np.rot90(np.rot90(currentObservation[:,:,0]))))
        plt.gca().invert_xaxis()
        plt.title('1')
        plt.show()
        
        plt.imshow(np.rot90(np.rot90(np.rot90(currentObservation[:,:,1]))))
        plt.gca().invert_xaxis()
        plt.title('2')
        plt.show()
        
        plt.imshow(np.rot90(np.rot90(np.rot90(currentObservation[:,:,2]))))
        plt.gca().invert_xaxis()
        plt.title('3')
        plt.show()
        
        plt.imshow(np.rot90(np.rot90(np.rot90(currentObservation[:,:,3]))))
        plt.gca().invert_xaxis()
        plt.title('4')
        plt.show()
        
    if timeStep % 10000 == 0:
        clear_output()
        
    timeStep += 1


步数 20001 Q值 [12.702301  5.721145]===> 步数 20001 / 本次行为 0 不跳 / 本次奖励 1 / Running   / 当前分数 540 / 最高得分 540
步数 20002 Q值 [11.890802   3.6896627]===> 步数 20002 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 540 / 最高得分 540
步数 20003 Q值 [11.911222  5.847076]===> 步数 20003 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 540 / 最高得分 540
步数 20004 Q值 [11.92584   8.128616]===> 步数 20004 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 540 / 最高得分 540
步数 20005 Q值 [11.998182  9.405185]===> 步数 20005 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 540 / 最高得分 540
步数 20006 Q值 [12.081442 10.428468]===> 步数 20006 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 540 / 最高得分 540
步数 20007 Q值 [12.062979 11.056842]===> 步数 20007 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 540 / 最高得分 540
步数 20008 Q值 [11.8852215 11.476195 ]===> 步数 20008 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 540 / 最高得分 540
步数 20009 Q值 [11.916529 11.804931]===> 步数 20009 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 540 / 最高得分 540
步数 20010 Q值 [11.843972 11.82062 ]===> 步数 20010 / 本次行为 0 不跳 / 本

步数 20082 Q值 [11.503841 11.802576]===> 步数 20082 / 本次行为 1 跳 / 本次奖励 0.1 / Running   / 当前分数 542 / 最高得分 542
步数 20083 Q值 [11.7408695 11.809279 ]===> 步数 20083 / 本次行为 1 跳 / 本次奖励 0.1 / Running   / 当前分数 542 / 最高得分 542
步数 20084 Q值 [11.830226 11.334008]===> 步数 20084 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 542 / 最高得分 542
步数 20085 Q值 [11.881809 11.194535]===> 步数 20085 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 542 / 最高得分 542
步数 20086 Q值 [11.911082 10.658907]===> 步数 20086 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 542 / 最高得分 542
步数 20087 Q值 [11.921282  10.4872265]===> 步数 20087 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 542 / 最高得分 542
步数 20088 Q值 [11.9675   10.768152]===> 步数 20088 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 542 / 最高得分 542
步数 20089 Q值 [12.011309 10.443287]===> 步数 20089 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 542 / 最高得分 542
步数 20090 Q值 [12.063097 10.177276]===> 步数 20090 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 542 / 最高得分 542
步数 20091 Q值 [12.149431  9.075857]===> 步数 20091 / 本次行为 0 不跳 / 本

步数 20164 Q值 [11.98838    9.4543495]===> 步数 20164 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 544 / 最高得分 544
步数 20165 Q值 [12.010115  8.123189]===> 步数 20165 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 544 / 最高得分 544
步数 20166 Q值 [11.956055   6.0159516]===> 步数 20166 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 544 / 最高得分 544
步数 20167 Q值 [11.955362   4.9913297]===> 步数 20167 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 544 / 最高得分 544
步数 20168 Q值 [12.045624  4.822132]===> 步数 20168 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 544 / 最高得分 544
步数 20169 Q值 [12.075995  4.263015]===> 步数 20169 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 544 / 最高得分 544
步数 20170 Q值 [12.002485   4.1140323]===> 步数 20170 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 544 / 最高得分 544
步数 20171 Q值 [12.109138   4.2638135]===> 步数 20171 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 544 / 最高得分 544
步数 20172 Q值 [12.069878  5.573474]===> 步数 20172 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 544 / 最高得分 544
步数 20173 Q值 [11.994296   5.6484427]===> 步数 20173 / 本次行

步数 20248 Q值 [12.045095 11.696861]===> 步数 20248 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 0 / 最高得分 545
步数 20249 Q值 [12.115068 11.737437]===> 步数 20249 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 0 / 最高得分 545
步数 20250 Q值 [12.194352 12.072921]===> 步数 20250 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 0 / 最高得分 545
步数 20251 Q值 [12.046527 12.069159]===> 步数 20251 / 本次行为 1 跳 / 本次奖励 0.1 / Running   / 当前分数 0 / 最高得分 545
步数 20252 Q值 [12.186955 11.973368]===> 步数 20252 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 0 / 最高得分 545
步数 20253 Q值 [12.291999 12.087274]===> 步数 20253 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 0 / 最高得分 545
步数 20254 Q值 [12.160608 11.405915]===> 步数 20254 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 0 / 最高得分 545
步数 20255 Q值 [12.177139  10.9777355]===> 步数 20255 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 0 / 最高得分 545
步数 20256 Q值 [12.076321 10.995543]===> 步数 20256 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 0 / 最高得分 545
步数 20257 Q值 [12.1322975 10.063469 ]===> 步数 20257 / 本次行为 0 不跳 / 本次奖励 0.1 / Running

步数 20328 Q值 [12.255354    0.11782614]===> 步数 20328 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 2 / 最高得分 545
步数 20329 Q值 [12.161035   1.0031445]===> 步数 20329 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 2 / 最高得分 545
步数 20330 Q值 [12.183953  2.321336]===> 步数 20330 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 2 / 最高得分 545
步数 20331 Q值 [12.14949    4.7985244]===> 步数 20331 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 2 / 最高得分 545
步数 20332 Q值 [12.207754  8.140284]===> 步数 20332 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 2 / 最高得分 545
步数 20333 Q值 [12.108362 10.461469]===> 步数 20333 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 2 / 最高得分 545
步数 20334 Q值 [12.301326 11.375536]===> 步数 20334 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 2 / 最高得分 545
步数 20335 Q值 [10.982326 12.042493]===> 步数 20335 / 本次行为 1 跳 / 本次奖励 0.1 / Running   / 当前分数 2 / 最高得分 545
步数 20336 Q值 [12.309812 10.467278]===> 步数 20336 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 2 / 最高得分 545
步数 20337 Q值 [12.302925  10.1936245]===> 步数 20337 / 本次行为 0 不跳 / 本次奖励 0.1 / R

步数 20411 Q值 [12.414704 10.255983]===> 步数 20411 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 4 / 最高得分 545
步数 20412 Q值 [12.475241  9.094427]===> 步数 20412 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 4 / 最高得分 545
步数 20413 Q值 [12.523555  7.570048]===> 步数 20413 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 4 / 最高得分 545
步数 20414 Q值 [12.495015   6.8115597]===> 步数 20414 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 4 / 最高得分 545
步数 20415 Q值 [12.477188   6.0685177]===> 步数 20415 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 4 / 最高得分 545
步数 20416 Q值 [12.460549  4.912819]===> 步数 20416 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 4 / 最高得分 545
步数 20417 Q值 [12.512962  4.082918]===> 步数 20417 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 4 / 最高得分 545
步数 20418 Q值 [12.553298   5.4869027]===> 步数 20418 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 4 / 最高得分 545
步数 20419 Q值 [12.552921   7.9003367]===> 步数 20419 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 4 / 最高得分 545
步数 20420 Q值 [12.546664  8.080788]===> 步数 20420 / 本次行为 0 不跳 / 本次奖励 1 / Runn

步数 20491 Q值 [12.50392    2.6630745]===> 步数 20491 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 6 / 最高得分 545
步数 20492 Q值 [12.584422   3.1789742]===> 步数 20492 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 6 / 最高得分 545
步数 20493 Q值 [12.545482  4.307369]===> 步数 20493 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 6 / 最高得分 545
步数 20494 Q值 [12.563396   5.1401186]===> 步数 20494 / 本次行为 0 不跳 / 本次奖励 1 / Running   / 当前分数 7 / 最高得分 545
步数 20495 Q值 [11.702692   4.9958997]===> 步数 20495 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 7 / 最高得分 545
步数 20496 Q值 [11.744244  8.309075]===> 步数 20496 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 7 / 最高得分 545
步数 20497 Q值 [11.819875 10.20948 ]===> 步数 20497 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 7 / 最高得分 545
步数 20498 Q值 [11.8332   10.745253]===> 步数 20498 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 7 / 最高得分 545
步数 20499 Q值 [11.880623 11.443592]===> 步数 20499 / 本次行为 0 不跳 / 本次奖励 0.1 / Running   / 当前分数 7 / 最高得分 545
步数 20500 Q值 [11.736559 11.93349 ]===> 步数 20500 / 本次行为 1 跳 / 本次奖励 0.1 / Runni